<a href="https://colab.research.google.com/github/laurarguezcorona/Calculadora-de-Bonos/blob/main/Tarea_Calculo_de_Volatilidades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Calculo de las volatilidades de Múltiples valores.
Calcule la volatilidad convencional (o fija en el tiempo), con suavizamiento exponencial con λ=0.95 , con suavizamiento exponencial con λ=0.98 , GARCH (1,1) gaussiano, GARCH (1,1) t-Student y GARCH (1,1) GED. Esto para los siguientes valores:

AC

ALSEA

ALFAA

BBAJIOO

BIMBOA

CEMEXCPO

CHDRAUI*

FUNO11

GENTERA*

GMEXICOB

GFINBURO

GFNORTEO

GMXT*

GRUMAB

KIMBERA

LACOMERUBC

PINFRA*

SORIANAB

WALMEXV

Reportar el cálculo para periodicidades diarias y semanales (históricos de 3 años) con pronósticos a t+1 y t+2 en los casos que amerite.

In [4]:
pip install yfinance arch pandas numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.9/982.9 kB 10.8 MB/s eta 0:00:00


In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
import matplotlib.pyplot as plt

In [6]:
# Listado de tickers
tickers = ["AC.MX", "ALSEA.MX", "ALFAA.MX", "BBAJIOO.MX", "BIMBOA.MX",
           "CEMEXCPO.MX", "CHDRAUIB.MX", "FUNO11.MX", "GENTERA.MX",
           "GMEXICOB.MX", "GFINBURO.MX", "GFNORTEO.MX", "GMXT.MX",
           "GRUMAB.MX", "KIMBERA.MX", "LACOMERUBC.MX", "PINFRA.MX",
           "SORIANAB.MX"]

In [7]:
# Descargar datos históricos de los últimos 3 años (diarios)
data = yf.download(tickers, start="2021-09-01", end="2024-09-01", interval='1d')['Adj Close']

[*********************100%***********************]  18 of 18 completed


In [8]:
print(data.head())

Ticker                          AC.MX   ALFAA.MX   ALSEA.MX  BBAJIOO.MX  \
Date                                                                      
2021-09-01 00:00:00+00:00  111.793579  12.775760  39.500919   27.296942   
2021-09-02 00:00:00+00:00  109.046425  12.812107  41.320557   27.219429   
2021-09-03 00:00:00+00:00  108.276535  12.612203  40.287788   26.979860   
2021-09-06 00:00:00+00:00  108.810204  12.594028  41.271378   27.578785   
2021-09-07 00:00:00+00:00  110.096298  12.766673  41.015648   27.614019   

Ticker                     BIMBOA.MX  CEMEXCPO.MX  CHDRAUIB.MX  FUNO11.MX  \
Date                                                                        
2021-09-01 00:00:00+00:00  48.671200    16.207167    30.161837  16.603796   
2021-09-02 00:00:00+00:00  48.402134    16.157177    30.482710  16.581270   
2021-09-03 00:00:00+00:00  47.479641    16.107185    30.434092  16.738974   
2021-09-06 00:00:00+00:00  48.017765    16.167175    30.677174  16.678890   
2021-09-07 0

In [9]:
# Calcular rendimientos diarios
returns = data.pct_change().dropna()

In [10]:
print(data.pct_change())

Ticker                        AC.MX  ALFAA.MX  ALSEA.MX  BBAJIOO.MX  \
Date                                                                  
2021-09-01 00:00:00+00:00       NaN       NaN       NaN         NaN   
2021-09-02 00:00:00+00:00 -0.024573  0.002845  0.046066   -0.002840   
2021-09-03 00:00:00+00:00 -0.007060 -0.015603 -0.024994   -0.008801   
2021-09-06 00:00:00+00:00  0.004929 -0.001441  0.024414    0.022199   
2021-09-07 00:00:00+00:00  0.011820  0.013709 -0.006196    0.001278   
...                             ...       ...       ...         ...   
2024-08-26 00:00:00+00:00  0.019991 -0.028789 -0.019996   -0.030742   
2024-08-27 00:00:00+00:00 -0.008560 -0.006975 -0.001806   -0.015354   
2024-08-28 00:00:00+00:00 -0.006050  0.005268  0.024783    0.003488   
2024-08-29 00:00:00+00:00  0.008812  0.013100 -0.013063    0.005520   
2024-08-30 00:00:00+00:00  0.020574 -0.010345 -0.032373   -0.002033   

Ticker                     BIMBOA.MX  CEMEXCPO.MX  CHDRAUIB.MX  FUNO11.MX  \

In [11]:
# Escalar los retornos por 100 para evitar problemas de convergencia
scaled = returns* 100
print(scaled.head())

Ticker                        AC.MX  ALFAA.MX  ALSEA.MX  BBAJIOO.MX  \
Date                                                                  
2021-09-02 00:00:00+00:00 -2.457345  0.284503  4.606569   -0.283961   
2021-09-03 00:00:00+00:00 -0.706020 -1.560278 -2.499405   -0.880139   
2021-09-06 00:00:00+00:00  0.492876 -0.144107  2.441408    2.219895   
2021-09-07 00:00:00+00:00  1.181961  1.370853 -0.619630    0.127759   
2021-09-08 00:00:00+00:00 -1.486008 -1.850528 -0.479622   -1.173778   

Ticker                     BIMBOA.MX  CEMEXCPO.MX  CHDRAUIB.MX  FUNO11.MX  \
Date                                                                        
2021-09-02 00:00:00+00:00  -0.552824    -0.308442     1.063839  -0.135666   
2021-09-03 00:00:00+00:00  -1.905893    -0.309408    -0.159495   0.951094   
2021-09-06 00:00:00+00:00   1.133379     0.372442     0.798716  -0.358943   
2021-09-07 00:00:00+00:00  -1.040622    -2.411879    -0.253561  -1.755952   
2021-09-08 00:00:00+00:00   0.647112    

In [32]:
# Función para aplicar GARCH y devolver resultados de diferentes distribuciones
def apply_garch_model(scaled, dist='normal'):
    model = arch_model(scaled, vol='Garch', p=1, q=1, dist=dist)
    model_fit = model.fit(disp="off")
    forecast = model_fit.forecast(horizon=2)
    return forecast.variance.iloc[-1, :].values  # Pronósticos de t+1 y t+2

In [33]:
# Inicializar DataFrame para resultados
scaled = pd.DataFrame(index=tickers, columns=['Convencional', 'Exp λ=0.95', 'Exp λ=0.98',
                                               'GARCH Normal t+1', 'GARCH t-Student t+1', 'GARCH GED t+1'])

In [34]:
print(results.head())

           Convencional Exp λ=0.95 Exp λ=0.98 GARCH Normal t+1  \
AC.MX          0.014119   1.429438   1.429927              NaN   
ALSEA.MX        0.01846   1.810614   1.809645              NaN   
ALFAA.MX       0.017898   1.789709   1.789738              NaN   
BBAJIOO.MX      0.01989   1.989536   1.989661              NaN   
BIMBOA.MX      0.020783   2.023203    2.02173              NaN   

           GARCH t-Student t+1 GARCH GED t+1  
AC.MX                      NaN           NaN  
ALSEA.MX                   NaN           NaN  
ALFAA.MX                   NaN           NaN  
BBAJIOO.MX                 NaN           NaN  
BIMBOA.MX                  NaN           NaN  


In [35]:
# Calcular volatilidades y aplicar modelos
for ticker in tickers:
    ticker_returns = returns[tickers].dropna()

In [36]:
print(ticker_returns.head())

Ticker                        AC.MX  ALSEA.MX  ALFAA.MX  BBAJIOO.MX  \
Date                                                                  
2021-09-02 00:00:00+00:00 -0.024573  0.046066  0.002845   -0.002840   
2021-09-03 00:00:00+00:00 -0.007060 -0.024994 -0.015603   -0.008801   
2021-09-06 00:00:00+00:00  0.004929  0.024414 -0.001441    0.022199   
2021-09-07 00:00:00+00:00  0.011820 -0.006196  0.013709    0.001278   
2021-09-08 00:00:00+00:00 -0.014860 -0.004796 -0.018505   -0.011738   

Ticker                     BIMBOA.MX  CEMEXCPO.MX  CHDRAUIB.MX  FUNO11.MX  \
Date                                                                        
2021-09-02 00:00:00+00:00  -0.005528    -0.003084     0.010638  -0.001357   
2021-09-03 00:00:00+00:00  -0.019059    -0.003094    -0.001595   0.009511   
2021-09-06 00:00:00+00:00   0.011334     0.003724     0.007987  -0.003589   
2021-09-07 00:00:00+00:00  -0.010406    -0.024119    -0.002536  -0.017560   
2021-09-08 00:00:00+00:00   0.006471    

In [46]:
# Volatilidad Convencional
scaled.loc[tickers, 'Convencional'] = np.std(ticker_returns)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [51]:
# Filtrar tickers problemáticos y calcular volatilidades
for ticker in tickers:
    scaled = ticker_returns[tickers].dropna()

    if len(ticker_returns) < 2:
        print(f"Insuficientes datos para el ticker {ticker}. Excluido.")
        continue  # Saltar si no hay suficientes datos

    try:
        # Volatilidad Convencional
        scaled.loc[ticker, 'Convencional'] = np.std(ticker_returns)
    except Exception as e:
        print(f"Error al calcular volatilidad convencional para el ticker {ticker_returns}: {e}")
        continue



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=ax

Error al calcular volatilidad convencional para el ticker Ticker                        AC.MX  ALSEA.MX  ALFAA.MX  BBAJIOO.MX  \
Date                                                                  
2021-09-02 00:00:00+00:00 -0.024573  0.046066  0.002845   -0.002840   
2021-09-03 00:00:00+00:00 -0.007060 -0.024994 -0.015603   -0.008801   
2021-09-06 00:00:00+00:00  0.004929  0.024414 -0.001441    0.022199   
2021-09-07 00:00:00+00:00  0.011820 -0.006196  0.013709    0.001278   
2021-09-08 00:00:00+00:00 -0.014860 -0.004796 -0.018505   -0.011738   
...                             ...       ...       ...         ...   
2024-08-26 00:00:00+00:00  0.019991 -0.019996 -0.028789   -0.030742   
2024-08-27 00:00:00+00:00 -0.008560 -0.001806 -0.006975   -0.015354   
2024-08-28 00:00:00+00:00 -0.006050  0.024783  0.005268    0.003488   
2024-08-29 00:00:00+00:00  0.008812 -0.013063  0.013100    0.005520   
2024-08-30 00:00:00+00:00  0.020574 -0.032373 -0.010345   -0.002033   

Ticker            

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [56]:
# Suavizamiento Exponencial (λ=0.95 y λ=0.98)
weights_95 = np.exp(np.linspace(-1., 0., len(ticker_returns)) * 0.95)
weights_95 /= weights_95.sum()
results.loc[tickers, 'Exp λ=0.95'] = np.sqrt(np.dot(weights_95, ticker_returns**2))

In [57]:
weights_98 = np.exp(np.linspace(-1., 0., len(ticker_returns)) * 0.98)
weights_98 /= weights_98.sum()
results.loc[tickers, 'Exp λ=0.98'] = np.sqrt(np.dot(weights_98, ticker_returns**2))

In [ ]:
# GARCH (1,1) para diferentes distribuciones
results.loc[tickers, 'GARCH Normal t+1'] = apply_garch_model(ticker_returns, 'normal')[0]
results.loc[tickers, 'GARCH t-Student t+1'] = apply_garch_model(ticker_returns, 't')[0]
results.loc[tickers, 'GARCH GED t+1'] = apply_garch_model(ticker_returns, 'ged')[0]

In [59]:
# Resaltar las columnas con colores diferentes para cada distribución
def color_map(val):
    if val.name == 'Convencional':
        return ['background-color: lightgreen'] * len(val)
    elif 'Exp' in val.name:
        return ['background-color: lightblue'] * len(val)
    elif 'GARCH Normal' in val.name:
        return ['background-color: lightcoral'] * len(val)
    elif 't-Student' in val.name:
        return ['background-color: lightyellow'] * len(val)
    elif 'GED' in val.name:
        return ['background-color: lightpink'] * len(val)
    return [''] * len(val)

In [60]:
# Aplicar el estilo de colores y mostrar
styled_results = results.style.apply(color_map)
styled_results.format("{:.5f}")  # Formatear decimales
styled_results

,Convencional,Exp λ=0.95,Exp λ=0.98,GARCH Normal t+1,GARCH t-Student t+1,GARCH GED t+1
AC.MX,0.01412,0.01429,0.01430,nan,nan,nan
ALSEA.MX,0.01846,0.01811,0.01810,nan,nan,nan
ALFAA.MX,0.01790,0.01790,0.01790,nan,nan,nan
BBAJIOO.MX,0.01989,0.01990,0.01990,nan,nan,nan
BIMBOA.MX,0.02078,0.02023,0.02022,nan,nan,nan
CEMEXCPO.MX,0.02265,0.02190,0.02188,nan,nan,nan
CHDRAUIB.MX,0.01640,0.01623,0.01622,nan,nan,nan
FUNO11.MX,0.01770,0.01757,0.01757,nan,nan,nan
GENTERA.MX,0.02224,0.02244,0.02245,nan,nan,nan
GMEXICOB.MX,0.02410,0.02362,0.02360,nan,nan,nan
